# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import requests
import json
from pprint import pprint



from scipy.stats import linregress
from matplotlib import pyplot as plt

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = "../WeatherPy/output_data/cities.csv"
citydf = pd.read_csv(data_file)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


locations = citydf[["Lat", "Lng"]].astype(float)


humid = citydf["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

humid_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(humid_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
citydf = pd.DataFrame(citydf.loc[citydf["Max Temp"] > 70, :])
citydf = pd.DataFrame(citydf.loc[citydf["Max Temp"] < 80, :])
citydf = pd.DataFrame(citydf.loc[citydf["Wind Speed"] < 10, :])
citydf = pd.DataFrame(citydf.loc[citydf["Cloudiness"] == 0, :])



citydf

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,Santa Fe,-31.63,-60.70,75.61,46,0,9.60,AR,1589649758
75,Al Bardīyah,31.76,25.09,75.96,54,0,8.16,LY,1589649764
101,Santa Maria,-29.68,-53.81,73.40,35,0,8.05,BR,1589649770
181,Camabatela,-8.19,15.38,70.32,61,0,4.21,AO,1589649790
291,Antalaha,-14.90,50.28,75.47,84,0,6.64,MG,1589649816
440,Alizai,33.54,70.35,74.17,31,0,4.59,PK,1589649851
541,Yima,34.74,111.88,78.01,99,0,3.13,CN,1589649875


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#hotel_df = pd.DataFrame(citydf.loc[citydf['City','Lat','Lng']])
hotel_df = citydf
hotel_df["Hotel Name"] = ""


params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"

    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    hotel_df.loc[index, "Hotel Name"] = response['results'][0]["name"]
    


In [7]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
51,Santa Fe,-31.63,-60.70,75.61,46,0,9.60,AR,1589649758,Conquistador Hotel
75,Al Bardīyah,31.76,25.09,75.96,54,0,8.16,LY,1589649764,بورتو بردية
101,Santa Maria,-29.68,-53.81,73.40,35,0,8.05,BR,1589649770,Hotel Morotin
181,Camabatela,-8.19,15.38,70.32,61,0,4.21,AO,1589649790,Pensão E Restaurantes
291,Antalaha,-14.90,50.28,75.47,84,0,6.64,MG,1589649816,Hotel Vitasoa


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
hotelmarkers = gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(hotelmarkers)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))